# Checking Cuda Environment

In [1]:
import torch
import os

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
print(f'Cuda version: {torch.version.cuda}')
device = torch.device("cuda")
print(f'n_cpu: {os.cpu_count()}')

Found device: GeForce GTX 1080 Ti, n_gpu: 1
Cuda version: 11.1
n_cpu: 64


# Loading Pre-trained GhostNet

In [2]:
import torch
from ghostnet import ghostnet

model = ghostnet(num_classes=1000, width=1.0, dropout=0.2)
input = torch.randn(32,3,224,224)
y = model(input)
print(y)

tensor([[ 1.2283e-01, -1.2960e-01,  3.5930e-02,  ...,  2.2548e-01,
         -2.5379e-02,  1.4319e-04],
        [ 7.5898e-02, -3.2058e-02, -1.5424e-02,  ...,  2.0955e-01,
         -5.4299e-02,  8.2759e-02],
        [ 6.2798e-02, -1.3268e-01,  8.6399e-02,  ...,  2.8293e-01,
          8.8400e-02,  1.1792e-01],
        ...,
        [ 1.0301e-01, -3.9318e-03,  4.2766e-02,  ...,  2.1013e-01,
          2.6327e-02,  1.4165e-02],
        [ 1.1281e-02, -4.9845e-02,  3.8997e-02,  ...,  1.6211e-01,
          3.6439e-02, -1.7583e-02],
        [ 7.9537e-02, -7.9260e-02, -2.0419e-02,  ...,  2.2794e-01,
         -6.5127e-02,  1.3466e-01]], grad_fn=<AddmmBackward>)


In [3]:
model.load_state_dict(torch.load('state_dict_73.98.pth'))
model = torch.nn.DataParallel(model, device_ids = list(range(n_gpu))).cuda()
model.eval()

DataParallel(
  (module): GhostNet(
    (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): GhostBottleneck(
          (ghost1): GhostModule(
            (primary_conv): Sequential(
              (0): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (cheap_operation): Sequential(
              (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
              (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
          )
          (ghost2): GhostModule(
            (primary_co

In [4]:
from PIL import Image
from torchvision import transforms
import torch

imsize = 224
loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])


def load_image(img_name):
    image = Image.open(img_name)
    image = loader(image).float()
    image = image.clone().detach().requires_grad_(True)
    image = image.unsqueeze(0)
    image = image.cuda()
    return image

# Tested on images from https://github.com/EliSchwartz/imagenet-sample-images
images = [load_image('ImageNetData/{0}.JPEG'.format(i)) for i in ['black_widow', 'magpie_0', 'magpie_1']]
image = images[0]
pred = model(image)
_, indices = torch.topk(pred, k=15)
print(indices)

tensor([[ 77,  73,  75,  70,  74,  72, 310, 315, 319,  79,  78, 312, 303,  76,
         120]], device='cuda:0')


# Changing Last Layer

In [5]:
import torch.nn as nn

num_classes = 8
num_features = model.module.classifier.in_features
new_last_layer = nn.Linear(num_features, num_classes).cuda()
model.module.classifier = new_last_layer

# Fine Tuning Model

In [6]:
from timm.data import create_dataset

dataset = create_dataset(name='', root='AffectNetData', transform=loader)
image, label = dataset[10000]
image.shape

torch.Size([3, 224, 224])

In [12]:
import torch
from torch.utils.data import DataLoader

batch_size = 64

train_size = int(0.8 * len(dataset))  # 80% for training
val_size = int(0.1 * len(dataset))   # 10% for validation
test_size = len(dataset) - train_size - val_size  # Remaining 10% for test

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size])

num_workers = 8
# pin_memory=True
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

In [14]:
from tqdm import tqdm
# import warnings 

# warnings.filterwarnings("ignore")

num_epochs = 450
loss_fn = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.RMSprop(model.parameters(), eps=0.001, lr=0.001, weight_decay=0.973)

model.train()
for epoch in range(num_epochs):
    total_samples = 0
    correct_predictions = 0
    total_loss = 0
    
    loop = tqdm(train_loader)
    for images, labels in loop:
        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels.cuda())
        
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels.cuda()).sum().item()
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        
        loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
        loop.set_postfix(loss=loss.item(), acc=(predicted == labels.cuda()).sum().item()/labels.size(0))
        
    avg_loss = total_loss / len(train_loader)
    avg_acc = correct_predictions / total_samples
    print("loss=", avg_loss, ", accuracy=", avg_acc)
    
    if epoch % 1 == 0:
        torch.save(model.state_dict(), f'ghostnet_checkpoints/checkpoint_{epoch}.pth')

  0%|          | 0/364 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch [0/450]: 100%|██████████| 364/364 [01:24<00:00,  4.30it/s, acc=0, loss=2.1]


loss= 2.0725066380186394 , accuracy= 0.1735462488701416


  0%|          | 0/364 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
Epoch [1/450]:  20%|█▉        | 72/364 [00:16<01:06,  4.38it/s, acc=0.234, loss=2.07]


KeyboardInterrupt: 